In [ ]:
import sys
sys.path.append("/opt/ml/input/code/pl")

from omegaconf import OmegaConf
from pytorch_lightning.callbacks import RichProgressBar

import warnings
warnings.filterwarnings(action='ignore')

from datamodule.base_data import *
from models.base_model import *

cfg = OmegaConf.load(f"/opt/ml/code/pl/config/roberta.yaml")

pl.seed_everything(cfg.train.seed, workers=True)

# dataloader와 model을 생성합니다.
dataloader = Dataloader(
    cfg.model.model_name,
    cfg.train.batch_size,
    cfg.data.shuffle,
    cfg.path.train_path,
    cfg.path.test_path,
    cfg.train.seed,
)
model = Model(cfg)

# gpu가 없으면 'gpus=0'을, gpu가 여러개면 'gpus=4'처럼 사용하실 gpu의 개수를 입력해주세요
trainer = pl.Trainer(
    precision=16,
    accelerator="gpu",
    devices=1,
    max_epochs=cfg.train.max_epoch,
    callbacks=[RichProgressBar()],
    auto_scale_batch_size="power"
    # deterministic=True,
    # limit_train_batches=0.15,  # use only 30% of training data
    # limit_val_batches = 0.01, # use only 1% of val data
    # limit_train_batches=10    # use only 10 batches of training data
)

new_batch_size = trainer.tuner.scale_batch_size(model=model,datamodule=dataloader)

print(new_batch_size)
